Square Cipher
=============

## Challenge
This is actually not a cipher, and I'm not quite sure what it has to do with a square?

```ncat --ssl square-cipher.challs.pwnoh.io 1337```

## Included files
* square_cipher.py

## Solution
The code provided is a bit hard to read, all on one line, let's clean it up

In [ ]:
def check(x):
    if (
        all(
            (
                int(bin(y).translate(str.maketrans('1b','fx')),0) & x
            ).bit_count()==15 
            for y in [511,261632,1838599,14708792,117670336,133955584,68585259008,35115652612096,246772580483072,1974180643864576,15793445150916608,17979214137393152,9205357638345293824,4713143110832790437888,4731607904558235517441,9463215809116471034882,18926431618232942069764,33121255085135066300416,37852863236465884139528,75705726472931768279056,151411452945863536558112,264970040681080530403328,302822905891727073116224,605645811783454146232448,1211291623566908292464896,2119760325448644243226624,2413129272746388704198656]
        ) 
        and x & 2135465562637171390290201561322170738230609084732268110734985633502584038857972308065155558608880 == 1271371190459412480076309932821732439054921890752535035282222258816851982409101952239053178406432 
    ):
        print(__import__('os').environ.get('FLAG','bctf{fake_flag}'))
    else:
        print('incorrect')
# check(int(input('> '),16))

At this point, I think this is certainly sat solvable. I haven't learned how to do this yet, so I'll use ChatGPT to do the hard work

In [26]:
from z3 import *
ys = [511,261632,1838599,14708792,117670336,133955584,68585259008,35115652612096,246772580483072,1974180643864576,15793445150916608,17979214137393152,9205357638345293824,4713143110832790437888,4731607904558235517441,9463215809116471034882,18926431618232942069764,33121255085135066300416,37852863236465884139528,75705726472931768279056,151411452945863536558112,264970040681080530403328,302822905891727073116224,605645811783454146232448,1211291623566908292464896,2119760325448644243226624,2413129272746388704198656]
masks = [int(bin(y).translate(str.maketrans('1b','fx')),0) for y in ys]

M2 = 2135465562637171390290201561322170738230609084732268110734985633502584038857972308065155558608880
C  = 1271371190459412480076309932821732439054921890752535035282222258816851982409101952239053178406432

W = max([m.bit_length() for m in masks] + [M2.bit_length(),C.bit_length()])

x = BitVec('x',W); 
s = Solver()

s.add((x & BitVecVal(M2,W)) == BitVecVal(C,W)) # x & M2 == C

for m in masks:
    bits = [(Extract(i,i,x & BitVecVal(m,W))==BitVecVal(1,1),1) for i in range(W) if (m>>i)&1]
    s.add(PbEq(bits,15))

assert s.check()==sat;
key = hex(s.model()[x].as_long())
print(key)

0x2986d08f14489753915af21287031754a0356c091cf48b118b246584a91c7078d942640f634687628


Repeated executions of the previous does give multiple values interestingly, but since we have a candidate, let's try it

In [28]:
from pwn import *

conn = remote("square-cipher.challs.pwnoh.io", 1337, ssl=True)
conn.sendline(key[2:])
print(conn.recvallS())

[x] Opening connection to square-cipher.challs.pwnoh.io on port 1337
[x] Opening connection to square-cipher.challs.pwnoh.io on port 1337: Trying 2600:1f16:75:1c01::4
[+] Opening connection to square-cipher.challs.pwnoh.io on port 1337: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 2B


/tmp/ipykernel_684155/939776062.py:4: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.sendline(key[2:])


[x] Receiving all data: 51B
[+] Receiving all data: Done (51B)
[*] Closed connection to square-cipher.challs.pwnoh.io port 1337
> bctf{5um_0f_f1r57_n_0dd_numb3r5_c1ph3r_025165aa}

